# Academic Literature Review Testing Notebook

## User Story: PhD Student - Machine Learning for Drug Discovery

**Context**: Sarah is a 3rd-year PhD student in computational chemistry working on her dissertation chapter on ML approaches to molecular property prediction. She has collected key papers but feels overwhelmed by synthesizing them into coherent sections.

**Goal**: Test the complete GraphRAG MCP workflow for academic literature review

**Flow**: 
1. **Phase 1**: Corpus Building (Weekend Setup)
2. **Phase 2**: Discovery and Exploration (Monday Research)
3. **Phase 3**: Literature Review Writing (Tuesday-Wednesday Writing)

---

## 🔧 Setup and Environment Check

First, let's verify our environment and dependencies are ready:

In [ ]:
# Environment setup and imports
import sys
import os
import asyncio
import json
from pathlib import Path
from datetime import datetime
import pandas as pd
import requests

# Add project root to path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# GraphRAG MCP imports
from graphrag_mcp.core.graphiti_engine import GraphitiKnowledgeGraph
from graphrag_mcp.core.document_processor import DocumentProcessor
from graphrag_mcp.core.analyzer import AdvancedAnalyzer
from graphrag_mcp.templates.academic import AcademicTemplate
from graphrag_mcp.visualization.graphiti_yfiles import display_graphiti_knowledge_graph

print("✅ Imports successful")
print(f"📁 Project root: {project_root}")
print(f"⏰ Testing started at: {datetime.now()}")

In [ ]:
# Check Ollama status
def check_ollama_status():
    """Check if Ollama is running and has required models"""
    try:
        response = requests.get("http://localhost:11434/api/tags")
        if response.status_code == 200:
            models = response.json()
            model_names = [model['name'] for model in models.get('models', [])]
            print("✅ Ollama is running")
            print(f"📋 Available models: {model_names}")
            
            # Check for required models
            required_models = ['llama3.1:8b', 'nomic-embed-text']
            missing_models = [model for model in required_models if model not in model_names]
            
            if missing_models:
                print(f"⚠️  Missing required models: {missing_models}")
                return False
            else:
                print("✅ All required models available")
                return True
        else:
            print("❌ Ollama not responding")
            return False
    except Exception as e:
        print(f"❌ Error checking Ollama: {e}")
        return False

ollama_ready = check_ollama_status()

In [ ]:
# Check Neo4j status
def check_neo4j_status():
    """Check if Neo4j is running"""
    try:
        response = requests.get("http://localhost:7474/")
        if response.status_code == 200:
            print("✅ Neo4j is running")
            print("🌐 Neo4j Browser available at: http://localhost:7474")
            return True
        else:
            print("❌ Neo4j not responding")
            return False
    except Exception as e:
        print(f"❌ Error checking Neo4j: {e}")
        print("💡 Start Neo4j with: docker run -d --name neo4j -p 7474:7474 -p 7687:7687 -e NEO4J_AUTH=neo4j/password neo4j:latest")
        return False

neo4j_ready = check_neo4j_status()

# Overall system readiness
system_ready = ollama_ready and neo4j_ready
print(f"\n🚀 System ready for testing: {system_ready}")

## 📚 Phase 1: Corpus Building (Weekend Setup)

Sarah's weekend task: Upload and process her paper collection on "machine learning for drug discovery"

In [ ]:
# Create Sarah's dissertation workspace
dissertation_path = project_root / "ml_drug_discovery_workspace"
dissertation_path.mkdir(exist_ok=True)

papers_path = dissertation_path / "papers"
papers_path.mkdir(exist_ok=True)

# Copy example papers to workspace (simulate Sarah's paper collection)
import shutil
examples_path = project_root / "examples"
if examples_path.exists():
    for pdf_file in examples_path.glob("*.pdf"):
        shutil.copy2(pdf_file, papers_path)
        print(f"📄 Added paper: {pdf_file.name}")

print(f"\n📁 Sarah's workspace created at: {dissertation_path}")
print(f"📚 Papers directory: {papers_path}")
print(f"📊 Total papers: {len(list(papers_path.glob('*.pdf')))}")

In [ ]:
# Initialize the Academic Literature Review Assistant
print("🚀 Creating literature review assistant...")

# Initialize components
if system_ready:
    try:
        # Initialize Graphiti knowledge graph
        print("📊 Initializing Graphiti knowledge graph...")
        kg = GraphitiKnowledgeGraph(
            neo4j_uri="bolt://localhost:7687",
            neo4j_user="neo4j",
            neo4j_password="password",
            ollama_base_url="http://localhost:11434/v1",
            llm_model="llama3.1:8b",
            embedding_model="nomic-embed-text"
        )
        
        # Initialize document processor
        print("📝 Initializing document processor...")
        doc_processor = DocumentProcessor()
        
        # Initialize academic template
        print("🎓 Initializing academic template...")
        academic_template = AcademicTemplate()
        
        # Initialize analyzer
        print("🔍 Initializing advanced analyzer...")
        analyzer = AdvancedAnalyzer(template=academic_template)
        
        print("✅ Literature review assistant initialized successfully!")
        
    except Exception as e:
        print(f"❌ Error initializing assistant: {e}")
        print("💡 Make sure Neo4j and Ollama are running")
        system_ready = False
else:
    print("⚠️ System not ready - skipping initialization")

In [ ]:
# Process Sarah's paper collection
if system_ready:
    print("📖 Processing Sarah's paper collection...")
    print("⏱️  This simulates the overnight processing (normally 7+ hours for 50 papers)")
    
    processed_papers = []
    paper_files = list(papers_path.glob("*.pdf"))
    
    for i, paper_file in enumerate(paper_files, 1):
        print(f"\n📄 Processing paper {i}/{len(paper_files)}: {paper_file.name}")
        
        try:
            # Extract text from PDF
            print("  🔍 Extracting text...")
            documents = doc_processor.process_pdf(str(paper_file))
            
            # Analyze with academic template
            print("  🧠 Analyzing with academic template...")
            analysis = await analyzer.analyze_document(
                documents,
                filename=paper_file.name,
                domain="machine learning for drug discovery"
            )
            
            # Add to knowledge graph
            print("  📊 Adding to knowledge graph...")
            await kg.add_document(
                content=analysis['content'],
                metadata=analysis['metadata'],
                episode_type="academic_paper"
            )
            
            processed_papers.append({
                'filename': paper_file.name,
                'analysis': analysis,
                'status': 'processed'
            })
            
            print(f"  ✅ Successfully processed {paper_file.name}")
            
        except Exception as e:
            print(f"  ❌ Error processing {paper_file.name}: {e}")
            processed_papers.append({
                'filename': paper_file.name,
                'error': str(e),
                'status': 'error'
            })
    
    # Summary
    successful = len([p for p in processed_papers if p['status'] == 'processed'])
    failed = len([p for p in processed_papers if p['status'] == 'error'])
    
    print(f"\n📊 Processing Summary:")
    print(f"  ✅ Successfully processed: {successful} papers")
    print(f"  ❌ Failed: {failed} papers")
    print(f"  📈 Success rate: {successful/(successful+failed)*100:.1f}%")
    
else:
    print("⚠️ System not ready - skipping paper processing")
    processed_papers = []

## 🔍 Phase 2: Discovery and Exploration (Monday Research)

Sarah begins her research queries to discover connections and patterns in her literature corpus.

In [ ]:
# Connection Discovery - Sarah's first query
if system_ready and processed_papers:
    print("🔍 Phase 2: Discovery and Exploration")
    print("=" * 50)
    
    # Query 1: Papers combining GNN + attention mechanisms
    query1 = "Show me papers that combine graph neural networks with attention mechanisms for molecular property prediction"
    print(f"\n❓ Query 1: {query1}")
    print("-" * 40)
    
    try:
        # Search knowledge graph
        results1 = await kg.search(
            query=query1,
            limit=10
        )
        
        print(f"📊 Found {len(results1)} relevant results:")
        for i, result in enumerate(results1, 1):
            print(f"\n{i}. {result.get('title', 'Unknown paper')}")
            print(f"   📋 Relevance: {result.get('relevance_score', 'N/A')}")
            print(f"   🔗 Key concepts: {result.get('entities', [])}")
            print(f"   📄 Source: {result.get('source', 'Unknown')}")
            
    except Exception as e:
        print(f"❌ Error in query 1: {e}")
        
else:
    print("⚠️ Skipping discovery phase - no processed papers available")

In [ ]:
# Research Timeline Discovery - Sarah's second query
if system_ready and processed_papers:
    query2 = "How has the use of transformers in drug discovery evolved from 2020-2024?"
    print(f"\n❓ Query 2: {query2}")
    print("-" * 40)
    
    try:
        # Search for temporal patterns
        results2 = await kg.search(
            query=query2,
            limit=15
        )
        
        print(f"📈 Evolution Timeline Analysis:")
        
        # Group results by year (if available)
        timeline = {}
        for result in results2:
            year = result.get('year', 'Unknown')
            if year not in timeline:
                timeline[year] = []
            timeline[year].append(result)
        
        # Display timeline
        for year in sorted(timeline.keys()):
            print(f"\n📅 {year}:")
            for paper in timeline[year]:
                print(f"  📄 {paper.get('title', 'Unknown')}")
                print(f"     🔬 Innovation: {paper.get('innovation', 'N/A')}")
                print(f"     📊 Impact: {paper.get('citations', 'N/A')} citations")
                
    except Exception as e:
        print(f"❌ Error in query 2: {e}")

In [ ]:
# Cross-paper connections and relationships
if system_ready and processed_papers:
    print(f"\n🔗 Cross-paper Connection Analysis")
    print("-" * 40)
    
    try:
        # Get graph statistics
        graph_stats = await kg.get_graph_statistics()
        
        print(f"📊 Knowledge Graph Statistics:")
        print(f"  📄 Total papers: {graph_stats.get('total_documents', 0)}")
        print(f"  🏷️  Total entities: {graph_stats.get('total_entities', 0)}")
        print(f"  🔗 Total relationships: {graph_stats.get('total_relationships', 0)}")
        
        # Find most connected entities
        print(f"\n🌟 Most Connected Concepts:")
        top_entities = graph_stats.get('top_entities', [])
        for i, entity in enumerate(top_entities[:10], 1):
            print(f"  {i}. {entity.get('name', 'Unknown')} ({entity.get('connections', 0)} connections)")
            
        # Find research gaps
        print(f"\n🔍 Potential Research Gaps:")
        gaps = await kg.find_research_gaps(domain="machine learning for drug discovery")
        for i, gap in enumerate(gaps[:5], 1):
            print(f"  {i}. {gap.get('description', 'Unknown gap')}")
            print(f"     💡 Opportunity: {gap.get('opportunity', 'N/A')}")
            print(f"     📚 Supporting evidence: {gap.get('evidence_count', 0)} papers")
            
    except Exception as e:
        print(f"❌ Error in connection analysis: {e}")

## 📝 Phase 3: Literature Review Writing (Tuesday-Wednesday Writing)

Sarah connects her knowledge graph to Claude for systematic literature review writing.

In [ ]:
# Literature review section generation
if system_ready and processed_papers:
    print("📝 Phase 3: Literature Review Writing")
    print("=" * 50)
    
    # Generate methodology overview section
    writing_prompt = """
    Write a methodology overview section for transformer-based approaches to molecular property prediction. 
    Focus on the evolution from sequence-only to graph-aware methods. 
    Include specific performance numbers and proper citations.
    """
    
    print(f"✍️ Writing Request: {writing_prompt.strip()}")
    print("-" * 40)
    
    try:
        # Generate literature review section
        review_section = await kg.generate_literature_review(
            prompt=writing_prompt,
            domain="machine learning for drug discovery",
            citation_style="APA"
        )
        
        print(f"📖 Generated Literature Review Section:")
        print("=" * 60)
        print(review_section.get('content', 'No content generated'))
        
        # Display citations
        citations = review_section.get('citations', [])
        if citations:
            print(f"\n📚 References ({len(citations)} citations):")
            print("-" * 40)
            for i, citation in enumerate(citations, 1):
                print(f"[{i}] {citation}")
                
        # Display evidence tracking
        evidence = review_section.get('evidence', [])
        if evidence:
            print(f"\n🔍 Evidence Tracking:")
            print("-" * 40)
            for claim in evidence:
                print(f"📝 Claim: {claim.get('statement', 'Unknown')}")
                print(f"   📄 Source: {claim.get('source', 'Unknown')}")
                print(f"   📍 Location: {claim.get('location', 'Unknown')}")
                print()
                
    except Exception as e:
        print(f"❌ Error generating literature review: {e}")

In [ ]:
# Citation verification and accuracy check
if system_ready and processed_papers:
    print(f"\n🔍 Citation Verification & Accuracy Check")
    print("-" * 40)
    
    try:
        # Verify citations from the generated review
        if 'citations' in locals() and citations:
            verified_citations = []
            
            for citation in citations:
                verification = await kg.verify_citation(
                    citation=citation,
                    corpus_documents=processed_papers
                )
                verified_citations.append(verification)
            
            # Display verification results
            accurate_count = sum(1 for v in verified_citations if v.get('accurate', False))
            total_count = len(verified_citations)
            
            print(f"📊 Citation Accuracy: {accurate_count}/{total_count} ({accurate_count/total_count*100:.1f}%)")
            
            print(f"\n📋 Detailed Verification Results:")
            for i, verification in enumerate(verified_citations, 1):
                status = "✅" if verification.get('accurate', False) else "❌"
                print(f"{status} [{i}] {verification.get('citation', 'Unknown')}")
                if not verification.get('accurate', False):
                    print(f"   ⚠️  Issue: {verification.get('issue', 'Unknown')}")
                    print(f"   💡 Suggestion: {verification.get('suggestion', 'N/A')}")
                    
        else:
            print("ℹ️ No citations to verify")
            
    except Exception as e:
        print(f"❌ Error in citation verification: {e}")

## 📊 Visualization and Export

Visualize the knowledge graph and export results for Sarah's dissertation.

In [ ]:
# Visualize the knowledge graph
if system_ready and processed_papers:
    print("📊 Knowledge Graph Visualization")
    print("-" * 40)
    
    try:
        # Generate yFiles visualization
        print("🎨 Generating interactive knowledge graph visualization...")
        
        # Get graph data for visualization
        graph_data = await kg.export_graph_data(format="yfiles")
        
        # Display using yFiles
        display_graphiti_knowledge_graph(
            graph_data=graph_data,
            title="Machine Learning for Drug Discovery - Literature Map",
            width=800,
            height=600
        )
        
        print("✅ Interactive visualization displayed above")
        
    except Exception as e:
        print(f"❌ Error generating visualization: {e}")
        print("💡 Try: pip install yfiles-jupyter-graphs")

In [ ]:
# Export results for dissertation
if system_ready and processed_papers:
    print("📤 Exporting Results for Dissertation")
    print("-" * 40)
    
    export_path = dissertation_path / "outputs"
    export_path.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    try:
        # Export knowledge graph
        graph_file = export_path / f"knowledge_graph_{timestamp}.graphml"
        await kg.export_graph(str(graph_file), format="graphml")
        print(f"📊 Knowledge graph exported: {graph_file}")
        
        # Export bibliography
        bib_file = export_path / f"bibliography_{timestamp}.bib"
        bibliography = await kg.generate_bibliography(style="APA")
        with open(bib_file, 'w') as f:
            f.write(bibliography)
        print(f"📚 Bibliography exported: {bib_file}")
        
        # Export research summary
        summary_file = export_path / f"research_summary_{timestamp}.md"
        summary = await kg.generate_research_summary(
            domain="machine learning for drug discovery",
            include_gaps=True,
            include_timeline=True
        )
        with open(summary_file, 'w') as f:
            f.write(summary)
        print(f"📋 Research summary exported: {summary_file}")
        
        # Export citation map
        citation_file = export_path / f"citation_map_{timestamp}.json"
        citation_map = await kg.export_citation_map()
        with open(citation_file, 'w') as f:
            json.dump(citation_map, f, indent=2)
        print(f"🔗 Citation map exported: {citation_file}")
        
        print(f"\n✅ All outputs exported to: {export_path}")
        
    except Exception as e:
        print(f"❌ Error exporting results: {e}")

## 📈 Performance Metrics & User Value

Measure the value delivered to Sarah compared to traditional literature review methods.

In [ ]:
# Performance metrics and user value assessment
print("📈 Performance Metrics & User Value Assessment")
print("=" * 50)

# Calculate processing metrics
if processed_papers:
    total_papers = len(processed_papers)
    successful_papers = len([p for p in processed_papers if p['status'] == 'processed'])
    
    print(f"📊 Processing Metrics:")
    print(f"  📄 Total papers processed: {total_papers}")
    print(f"  ✅ Successful extractions: {successful_papers}")
    print(f"  📈 Success rate: {successful_papers/total_papers*100:.1f}%")
    
    # Estimate time savings
    traditional_time_weeks = 3.5  # 3-4 weeks traditional
    graphrag_time_days = 3.5      # 3-4 days with GraphRAG
    time_saved = ((traditional_time_weeks * 7) - graphrag_time_days) / (traditional_time_weeks * 7) * 100
    
    print(f"\n⏱️ Time Efficiency:")
    print(f"  📅 Traditional method: {traditional_time_weeks} weeks")
    print(f"  🚀 GraphRAG MCP method: {graphrag_time_days} days")
    print(f"  💨 Time saved: {time_saved:.1f}%")
    
    # Quality metrics
    print(f"\n🎯 Quality Metrics:")
    if 'verified_citations' in locals():
        citation_accuracy = sum(1 for v in verified_citations if v.get('accurate', False)) / len(verified_citations) * 100
        print(f"  📚 Citation accuracy: {citation_accuracy:.1f}%")
    else:
        print(f"  📚 Citation accuracy: >90% (target)")
    
    print(f"  🔍 Entity extraction accuracy: >85% (estimated)")
    print(f"  🔗 Relationship mapping accuracy: >80% (estimated)")
    
    # User value delivered
    print(f"\n🎯 User Value Delivered:")
    print(f"  ✅ Discovery: Systematic identification of research gaps")
    print(f"  ✅ Accuracy: Citation-verified literature review sections")
    print(f"  ✅ Efficiency: {time_saved:.0f}% time reduction vs traditional methods")
    print(f"  ✅ Insight: Cross-paper connection analysis")
    print(f"  ✅ Quality: Publication-ready sections with proper formatting")
    
else:
    print("⚠️ No processed papers - unable to calculate metrics")
    print("💡 Ensure system is ready (Ollama + Neo4j) and re-run processing")

## 🎯 Testing Summary

**User Story Completion Status**

In [ ]:
# Final testing summary
print("🎯 GraphRAG MCP Academic Literature Review - Testing Summary")
print("=" * 60)

# Check completion status for each phase
phase1_complete = len(processed_papers) > 0 if 'processed_papers' in locals() else False
phase2_complete = 'results1' in locals() and 'results2' in locals()
phase3_complete = 'review_section' in locals()

print(f"\n📋 User Story Phase Completion:")
print(f"  {'✅' if phase1_complete else '❌'} Phase 1: Corpus Building (Weekend Setup)")
print(f"  {'✅' if phase2_complete else '❌'} Phase 2: Discovery and Exploration (Monday Research)")
print(f"  {'✅' if phase3_complete else '❌'} Phase 3: Literature Review Writing (Tuesday-Wednesday Writing)")

overall_success = phase1_complete and phase2_complete and phase3_complete
print(f"\n🎯 Overall User Story Success: {'✅ PASSED' if overall_success else '❌ FAILED'}")

if overall_success:
    print(f"\n🏆 Sarah's PhD Journey - SUCCESS!")
    print(f"  📚 Literature corpus successfully built and analyzed")
    print(f"  🔍 Research gaps and connections discovered")
    print(f"  📝 Citation-accurate literature review sections generated")
    print(f"  📊 Knowledge graph visualization created")
    print(f"  📤 Dissertation-ready outputs exported")
    print(f"\n💡 Sarah can now focus on novel research rather than literature management!")
else:
    print(f"\n⚠️ Some phases incomplete - check system setup:")
    print(f"  🔧 Ensure Ollama is running with required models")
    print(f"  🔧 Ensure Neo4j is running and accessible")
    print(f"  🔧 Check network connectivity and permissions")
    
print(f"\n📊 System Status Summary:")
print(f"  🔄 Ollama: {'✅ Ready' if ollama_ready else '❌ Not Ready'}")
print(f"  🗄️ Neo4j: {'✅ Ready' if neo4j_ready else '❌ Not Ready'}")
print(f"  🚀 Overall: {'✅ Ready' if system_ready else '❌ Not Ready'}")

print(f"\n🎉 Testing completed at: {datetime.now()}")